In [73]:
import requests
import pandas as pd

def hit_gov_data(start_date='2001-01', end_date='2023-01', offset=0, length=5000, api_key="YFxmf9XXnim4IBQnrbFuV2QgJDObwYaxzEabiH4b"):

    api_url = f"https://api.eia.gov/v2/electricity/retail-sales/data/?api_key={api_key}&frequency=monthly&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&start={start_date}&end={end_date}&sort[0][column]=period&sort[0][direction]=desc&offset={offset}&length={length}"
    # api_url = "https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
    response = requests.get(api_url)
    if response.status_code == 200:
        return pd.DataFrame(response.json()['response']['data'])
    else:
        raise Exception("ISSUE HITTING US GOVERMENT DATA API")

In [74]:
df = hit_gov_data(offset=0, length=5000, end_date='2024-01')

In [75]:
df.isna().sum()

period                0
stateid               0
stateDescription      0
sectorid              0
sectorName            0
customers           833
price               833
revenue             833
sales               833
customers-units       0
price-units           0
revenue-units         0
sales-units           0
dtype: int64

In [76]:
df[(df['stateid']=='MD') & (df['sectorid']=='RES')]

,period,stateid,stateDescription,sectorid,sectorName,customers,price,revenue,sales,customers-units,price-units,revenue-units,sales-units
0,2023-09,MD,Maryland,RES,residential,2431160.0,16.96,375.82097,2215.58311,number of customers,cents per kilowatthour,million dollars,million kilowatthours
663,2023-08,MD,Maryland,RES,residential,2429658.0,16.35,431.98451,2642.54595,number of customers,cents per kilowatthour,million dollars,million kilowatthours
806,2023-07,MD,Maryland,RES,residential,2428046.0,16.03,463.06933,2887.89109,number of customers,cents per kilowatthour,million dollars,million kilowatthours
1153,2023-06,MD,Maryland,RES,residential,2427170.0,16.99,334.54401,1969.06761,number of customers,cents per kilowatthour,million dollars,million kilowatthours
1786,2023-05,MD,Maryland,RES,residential,2426004.0,16.08,267.50220,1663.98981,number of customers,cents per kilowatthour,million dollars,million kilowatthours
2083,2023-04,MD,Maryland,RES,residential,2425944.0,15.91,274.87139,1727.16111,number of customers,cents per kilowatthour,million dollars,million kilowatthours
2578,2023-03,MD,Maryland,RES,residential,2426467.0,15.85,348.56671,2199.67197,number of customers,cents per kilowatthour,million dollars,million kilowatthours
2936,2023-02,MD,Maryland,RES,residential,2423960.0,16.21,357.64479,2206.17096,number of customers,cents per kilowatthour,million dollars,million kilowatthours
3270,2023-01,MD,Maryland,RES,residential,2421516.0,15.97,401.98277,2516.41400,number of customers,cents per kilowatthour,million dollars,million kilowatthours
3656,2022-12,MD,Maryland,RES,residential,2424156.0,15.53,436.30499,2809.69318,number of customers,cents per kilowatthour,million dollars,million kilowatthours


In [77]:
df['sales-units'].value_counts()

sales-units
million kilowatthours    5000
Name: count, dtype: int64

In [78]:
def separate_dimensionality(df):
    frame = df.copy()
    frame = frame.drop(['customers-units', 'price-units', 'revenue-units', 'sales-units'], axis=1)
    metrics = df[['customers-units', 'price-units', 'revenue-units', 'sales-units']]
    metrics = metrics.head(1).to_dict(orient='records')[0]
    return metrics, frame
    

In [79]:
metrics, df = separate_dimensionality(df)

In [81]:
metrics

{'customers-units': 'number of customers',
 'price-units': 'cents per kilowatthour',
 'revenue-units': 'million dollars',
 'sales-units': 'million kilowatthours'}